In [1]:
import json
import glob
import pandas as pd
from pandas import json_normalize

from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score #average_precision_score

In [2]:
pd.__version__

'1.0.3'

In [3]:
# import all json --> pandas df

In [5]:
# #test_files = "translate_testfive/Homo_sapiens_cdna_abinitio_04_dayhoff_k9_refHsapiens_QfO_t1e8.json"
# test_file = "translate_testfive/Homo_sapiens_ncRNA_05_protein_k9_refsprot_t1e9.json"

# with open(test_file) as f:
#   testdata = json.load(f)

In [6]:
testdata

NameError: name 'testdata' is not defined

In [7]:
testdf = json_normalize(testdata)
testdf.head()

NameError: name 'testdata' is not defined

In [ ]:
def build_summary_df(filename_list):
    all_info=[]
    for fi in filename_list:
        with open(fi) as f:
            all_info.append(json_normalize(json.load(f)))
    return all_info
            
import glob
#ncRNA = glob.glob("translate_testfive/*ncRNA_01*json")
#qfo_dna = glob.glob("translate_testfive/*qfo_dna_01*json")
#ncRNA = glob.glob("translate_testfive/*ncRNA*json")
#qfo_dna = glob.glob("translate_testfive/*qfo_dna*json")
#ncRNA = glob.glob("translate_Hsapiens/*ncRNA*json")
#qfo_dna = glob.glob("translate_Hsapiens/*qfo_dna*json")

#ncRNA = glob.glob("translate_s100/Homo_sapiens*ncRNA*json")
#qfo_dna = glob.glob("translate_s100/Homo_sapiens*qfo_dna*json")

ncRNA = glob.glob("translate_s100/*ncRNA*json")
qfo_dna = glob.glob("translate_s100/*qfo_dna*json")

filename_list = ncRNA + qfo_dna

In [ ]:
len(filename_list)

In [ ]:
summaryInfo = build_summary_df(filename_list)

In [ ]:
summaryDF = pd.concat(summaryInfo).fillna(0)

In [ ]:
summaryDF.head()

In [ ]:
summaryDF.columns

In [ ]:
summaryDF.head()

In [ ]:
#summaryDF[["input_files", "peptide_alphabet", "peptide_ksize", 'categorization_percentages.Coding', 'categorization_percentages.Non-coding']]

In [ ]:
summaryDF["total_reads"] = summaryDF.groupby((summaryDF.columns.str.split(".").str[0].str.contains("categorization_counts")),axis=1).sum()[True]

In [ ]:
summaryDF["categorization_percentages.Coding"].tail()

In [ ]:
summaryDF["categorization_percentages.Non-coding"].head()

In [ ]:
#A true positive is an outcome where the model correctly predicts the positive class. 
#Similarly, a true negative is an outcome where the model correctly predicts the negative class
#A false positive is an outcome where the model incorrectly predicts the positive class.
#And a false negative is an outcome where the model incorrectly predicts the negative class.
# positive class = "coding"
# TP = "coding" FP = "total" - "coding"
# negative class = "non-coding"
# TN = "non-coding" FN= "total" - "non-coding" except NOT. Actually = "total - coding"
# bc there are other classes, but it really is coding = coding, everything else = non-coding
import re

def find_origin(row):
    if "qfo" in row["input_files"][0]:
        row["origin"] = "coding"
        row["true_positives"] = row["categorization_counts.Coding"]
        row["true_negatives"] = 0
        row["false_positives"] = 0
        row["false_negatives"] = row["total_reads"] - row["categorization_counts.Coding"]
        input_file = row["input_files"][0]
        row["species"] = re.search("(\w*)_qfo", input_file).groups()[0].replace("_", " ")
    elif "ncRNA" in row["input_files"][0]:
        row["origin"] = "noncoding"
        row["true_positives"] = 0
        row["true_negatives"] = row["total_reads"] - row["categorization_counts.Coding"]
        row["false_positives"] = row["categorization_counts.Coding"]
        row["false_negatives"]= 0
        input_file = row["input_files"][0]
        row["species"] = re.search("(\w*)_ncRNA", input_file).groups()[0].replace("_", " ")
        #row["species"] = row["input_files"].str.extract("(\w*)_ncRNA")[0]
    if "Hsapiens" in row["peptide_bloom_filter"]:
        row["peptide_reference"] = "Homo sapiens QfO"
    elif "sprot" in row["peptide_bloom_filter"]:
        row["peptide_reference"] = "Swiss Prot"
    return row

summaryDF = summaryDF.apply(find_origin, axis=1)

In [ ]:
#now we should add divergence time
#species_metadata= https://docs.google.com/spreadsheets/d/1dCT8IXTdlAuz7mng97x3T5vxiY5KllbYYXFNzP0H0Vc/edit#gid=1807612802
species_metadata = pd.read_csv("species_metadata.csv")
#dtime = pd.read_csv("divergence_time.txt")

In [ ]:
species_metadata.head()

In [ ]:
species_metadata[["scientific_name", "divergence_from_human_mya"]]

In [ ]:
summaryDF=summaryDF.merge(species_metadata[["scientific_name", "divergence_from_human_mya"]], left_on=["species"], right_on=["scientific_name"])

In [ ]:
summaryDF.head()

In [ ]:
summaryDF.iloc[0]["input_files"][0]

In [ ]:
summaryDF[["input_files", "origin","species", "divergence_from_human_mya", "peptide_alphabet", "peptide_ksize", 'categorization_percentages.Coding', 'categorization_percentages.Non-coding', "true_positives"]].head()

In [ ]:
# let's check - for *01* files, there should be two files in each group (one from coding reads, one from noncoding)
# if including replicates (5) -- should be 10 files in each group
# actually probably want to compute metrics INDEPENDENTLY for each replicate set
summaryDF.groupby(["species", "divergence_from_human_mya", "peptide_alphabet", "peptide_ksize", "peptide_bloom_filter", "jaccard_threshold"]).count()

In [ ]:
# want to combine values of true pos, true neg, false pos, false neg values
#new multindex dataframe with true pos, true neg, false pos, false neg columns, precision, recall, f1 score


In [ ]:
metricDF = summaryDF.groupby(["species", "divergence_from_human_mya", "peptide_alphabet", "peptide_reference", "peptide_ksize", "peptide_bloom_filter","jaccard_threshold"]).sum()[["true_positives", "true_negatives", "false_positives", "false_negatives"]]

In [ ]:
summaryDF.loc[(summaryDF["peptide_alphabet"]=="dayhoff") & (summaryDF["peptide_ksize"]==13)]["categorization_percentages.Coding"]

# first read: 54% non-coding, 11% coding, 35% other. might need to keep other categories for good f1 score?

In [ ]:
# look at numbers per bloom filter
#summaryDF.groupby(["peptide_bloom_filter"]).sum()[["true_positives", "true_negatives", "false_positives", "false_negatives"]]

In [ ]:
# precision = TP/ (TP + FP)
# recall = TP/ (TP + FN)
# F1 = 2 * (precision * recall) / (precision + recall)

In [ ]:
metricDF["precision"] = metricDF["true_positives"]/(metricDF["true_positives"] + metricDF["false_positives"])
metricDF["recall"] = metricDF["true_positives"]/(metricDF["true_positives"] + metricDF["false_negatives"])
metricDF["F1_score"] = 2 * ((metricDF["precision"] * metricDF["recall"]) / (metricDF["precision"] + metricDF["recall"]))

In [ ]:
metricDF.head()

In [ ]:
print("max_f1score: " + str(metricDF["F1_score"].max()))
print("min_f1score: " + str(metricDF["F1_score"].min()))
print("max_precision: " + str(metricDF["precision"].max()))
print("min_precision: " + str(metricDF["precision"].min()))
print("max_recall: " + str(metricDF["recall"].max()))
print("min_recall: " + str(metricDF["recall"].min()))

In [ ]:
# NOW, PLOTTING!

In [ ]:
plotDF = metricDF.reset_index()

In [ ]:
plotDF.to_csv("QfO_vertebrates_s100_sencha_translate_summaries.csv")

In [ ]:
#plotDF["peptide_reference"] = plotDF["peptide_bloom_filter"].str.extract

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="white")

In [ ]:

HsapiensDF = plotDF[(plotDF["species"] == "Homo sapiens")]
sns.scatterplot(x="peptide_ksize", y="precision", data=HsapiensDF, hue="peptide_alphabet" )#, style="jaccard_threshold", legend="full")

In [ ]:
sns.lineplot(x="peptide_ksize", y="precision", data=HsapiensDF, hue="peptide_alphabet")#, style="jaccard_threshold", legend="full")

In [ ]:
sns.lineplot(x="peptide_ksize", y="precision", data=plotDF[(plotDF["species"] == "Bos_taurus")], hue="peptide_alphabet")#, style="jaccard_threshold", legend="full")

In [ ]:
#g = sns.FacetGrid(plotDF, row="peptide_reference", col="jaccard_threshold", hue="peptide_alphabet", legend_out= True, height=6, aspect=1) #, style="jaccard_threshold"
g = sns.FacetGrid(plotDF, row="peptide_reference", col="peptide_alphabet", hue="jaccard_threshold", legend_out= True, height=6, aspect=1) #, style="jaccard_threshold"
g = g.map(plt.scatter, "peptide_ksize", "precision", edgecolor="w")

In [ ]:
sns.scatterplot(x="peptide_ksize", y="precision", data=plotDF.loc[plotDF["peptide_reference"] == "Homo sapiens QfO"], hue="peptide_alphabet")#, style="jaccard_threshold", legend=False)

In [ ]:
sns.scatterplot(x="peptide_ksize", y="precision", data=plotDF.loc[plotDF["peptide_reference"] == "Swiss Prot"], hue="peptide_alphabet")#, style="jaccard_threshold", legend=False)


In [ ]:
# let's grab just default thresholds
#df[df['first_name'].notnull() & (df['nationality'] == "USA")]
#defaultThresholds=
proteinDF = plotDF[(plotDF["jaccard_threshold"] == 0.5) & (plotDF['peptide_alphabet'] =="protein")]
dayhoffDF = plotDF[(plotDF["jaccard_threshold"] == 0.5) & (plotDF['peptide_alphabet'] =="dayhoff")]
hpDF = plotDF[(plotDF["jaccard_threshold"] == 0.8) & (plotDF['peptide_alphabet'] =="hydrophobic-polar")]
defaultThresholds = pd.concat([proteinDF, dayhoffDF, hpDF])


#in ["protein", "dayhoff"])]
#dfObj[(dfObj['Sale'] > 30) & (dfObj['Sale'] < 33) ]


In [ ]:
defaultThresholds

In [ ]:
hpDF

In [ ]:
sns.lineplot(x="recall", y="precision", data=defaultThresholds, hue="peptide_alphabet")#, style="jaccard_threshold", legend="full")

In [ ]:
#g = sns.FacetGrid(plotDF, row="peptide_reference", col="jaccard_threshold", hue="peptide_alphabet", legend_out= True, height=6, aspect=1) #, style="jaccard_threshold"
#g = g.map(plt.plot, "peptide_ksize", "precision", edgecolor="w")

In [ ]:
g = sns.FacetGrid(defaultThresholds, row= "species", col="peptide_reference", hue="peptide_alphabet", legend_out= True, height=6, aspect=1) #, style="jaccard_threshold"
#g = g.map(plt.scatter, "peptide_ksize", "precision", edgecolor="w")
g = g.map(plt.plot, "peptide_ksize", "precision") #, edgecolor="w")

In [ ]:
#defaultThresholds=plotDF.loc[plotDF["jaccard_threshold"] == 0.5]
#g = sns.FacetGrid(defaultThresholds,col="peptide_ksize", row= "peptide_reference",height=6, aspect=1)#, height=2)
g = sns.FacetGrid(plotDF,col="peptide_ksize", row= "peptide_reference",height=6, aspect=1)
g = g.map(plt.plot, "recall", "precision", marker=".")
# weird patterns are a result of having all thresholds in same plot - need to break out

In [ ]:
#F1 score vs peptide_ksize
sns.lineplot(x="peptide_ksize", y="F1_score", data=plotDF, hue="peptide_alphabet")#, style="jaccard_threshold", legend="full")

In [ ]:
Hsapiens_ref = plotDF.loc[plotDF["peptide_reference"] == "Homo sapiens QfO"]
Sprot_ref = plotDF.loc[plotDF["peptide_reference"] == "Swiss Prot"]
#defaultThresholds=plotDF.loc[plotDF["jaccard_threshold"] == 0.5]
sns.lineplot(x="peptide_ksize", y="F1_score", data=Hsapiens_ref, hue="peptide_alphabet")#, style="jaccard_threshold", legend="full")

In [ ]:
Sprot_ref = plotDF.loc[plotDF["peptide_reference"] == "Swiss Prot"]
#defaultThresholds=plotDF.loc[plotDF["jaccard_threshold"] == 0.5]
sns.lineplot(x="peptide_ksize", y="F1_score", data=Sprot_ref, hue="peptide_alphabet")#, style="jaccard_threshold", legend="full")

In [ ]:
# to do: better select-threshold
sns.lineplot(x="peptide_ksize", y="F1_score", data=defaultThresholds, hue="peptide_alphabet")#, style="jaccard_threshold", legend="full")

In [ ]:
# this isn't very informative - would need rate (%) though here I know total for each - (coding: 500, nc: 500)
sns.lineplot(x="false_positives", y="true_positives", data=Sprot_ref, hue="peptide_alphabet")

In [ ]:
defaultThresholds

In [ ]:
sns.lineplot(x="divergence_from_human_mya", y="F1_score", data=defaultThresholds[defaultThresholds["peptide_reference"]=="Homo sapiens QfO"], hue="peptide_alphabet")

In [ ]:
sns.lineplot(x="divergence_from_human_mya", y="precision", data=defaultThresholds[defaultThresholds["peptide_reference"]=="Homo sapiens QfO"], hue="peptide_alphabet")

In [ ]:
sns.lineplot(x="divergence_from_human_mya", y="recall", data=defaultThresholds[defaultThresholds["peptide_reference"]=="Homo sapiens QfO"], hue="peptide_alphabet")

In [ ]:
sns.lineplot(x="divergence_from_human_mya", y="F1_score", data=defaultThresholds[defaultThresholds["peptide_reference"]=="Swiss Prot"], hue="peptide_alphabet")

In [ ]:
sns.lineplot(x="divergence_from_human_mya", y="precision", data=defaultThresholds[defaultThresholds["peptide_reference"]=="Swiss Prot"], hue="peptide_alphabet")

In [ ]:
sns.lineplot(x="divergence_from_human_mya", y="recall", data=defaultThresholds[defaultThresholds["peptide_reference"]=="Swiss Prot"], hue="peptide_alphabet")

In [ ]:
g = sns.FacetGrid(defaultThresholds[defaultThresholds["peptide_reference"]=="Homo sapiens QfO"],col="peptide_alphabet")
#g = g.map(plt.plot, "divergence_from_human_mya","F1_score", marker=".")
g = g.map(plt.scatter, "divergence_from_human_mya", "F1_score", edgecolor="w")#, style="peptide_ksize")

In [ ]:
defaultThresholds

In [ ]:
#df.loc[df.reset_index().groupby(['F_Type'])['to_date'].idxmax()]
#bestF1 = defaultThresholds[defaultThresholds.groupby(["species"])["F1_score"]].idxmax()]

bestF1=defaultThresholds.loc[defaultThresholds.groupby(["species", "divergence_from_human_mya", "peptide_alphabet", "peptide_reference"])["F1_score"].idxmax()]

In [ ]:
#bF1=bestF1.reset_index()

In [ ]:
bestF1

In [ ]:
sns.lineplot(data=bestF1.loc[plotDF["peptide_reference"] == "Homo sapiens QfO"], x="peptide_ksize", y="F1_score",hue="peptide_alphabet")

In [ ]:
sns.pointplot(data=bestF1.loc[bestF1["peptide_reference"] == "Homo sapiens QfO"], x="peptide_ksize", y="F1_score",hue="peptide_alphabet", edgecolor="w")

In [ ]:
g = sns.FacetGrid(bestF1[bestF1["peptide_reference"]=="Homo sapiens QfO"],col="divergence_from_human_mya")

#g = g.map(plt.plot, "peptide_ksize","F1_score", marker=".")
g = g.map(plt.scatter, "peptide_ksize", "F1_score", edgecolor="w")#, style="peptide_ksize")

In [ ]:
g = sns.catplot(x="peptide_ksize", y="F1_score",hue="peptide_alphabet", col="divergence_from_human_mya",data=bestF1[bestF1["peptide_reference"]=="Homo sapiens QfO"], kind="point", dodge=True, height=4, aspect=1);